# Données WFP - Admin 1

In [ ]:
import sys
import os

# Détection automatique de la racine du projet
# On remonte d'un niveau depuis le dossier 'notebooks'
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Ajout au path pour pouvoir faire "from src..."
if project_root not in sys.path:
    sys.path.append(project_root)

# Définition des chemins absolus
DATA_RAW = os.path.join(project_root, "data", "raw")
DATA_PROCESSED = os.path.join(project_root, "data", "processed")
GEO_PATH = os.path.join(project_root, "geo")

print(f"Système configuré. Racine : {project_root}")

In [10]:
import pandas as pd
import os

# ==============================================================================
# 1. CONFIGURATION DES CHEMINS
# ==============================================================================
# Ton fichier météo (généré à l'étape précédente)
fldas_path = "../data/raw/FLDAS_Nigeria_Admin2_Aggregated.csv" 
# Le fichier WFP 
wfp_path = "../data/raw/WFP_food_security_data_nigeria.csv"
# Fichier de sortie
output_path = "../data/processed/NIGERIA_WFP_ADMIN1.parquet"


In [11]:

# ==============================================================================
# 2. CHARGEMENT ET PRÉPARATION FLDAS (CLIMAT)
# ==============================================================================
if os.path.exists(fldas_path):
    print("Chargement des données climatiques (Admin2)...")
    df_climate = pd.read_csv(fldas_path)
    df_climate['time'] = pd.to_datetime(df_climate['time'])
    
    # --- L'ÉTAPE CRUCIALE : AGRÉGATION ADMIN2 -> ADMIN1 ---
    # On fait la moyenne de toutes les LGA pour obtenir une valeur par État
    print("   -> Agrégation par État (Admin1)...")
    df_climate_admin1 = df_climate.groupby(['time', 'NAME_1']).mean(numeric_only=True).reset_index()
    
    # Petit nettoyage des noms d'états pour que ça matche (FCT / Abuja)
    # Dans GADM c'est souvent "Federal Capital Territory", dans WFP c'est "Abuja"
    df_climate_admin1['NAME_1'] = df_climate_admin1['NAME_1'].replace({
        'Federal Capital Territory': 'Abuja',
        'Nassarawa': 'Nasarawa' # Cas fréquent de faute de frappe
    })
    
    print(f"   -> Climat prêt : {df_climate_admin1.shape} lignes.")
else:
    raise FileNotFoundError(f" Impossible de trouver {fldas_path}. Lance le calcul précédent !")


Chargement des données climatiques (Admin2)...
   -> Agrégation par État (Admin1)...
   -> Climat prêt : (2220, 30) lignes.


In [12]:

# ==============================================================================
# 3. CHARGEMENT ET PRÉPARATION WFP (CIBLE)
# ==============================================================================
if os.path.exists(wfp_path):
    print("Chargement des données WFP...")
    df_wfp = pd.read_csv(wfp_path)
    
    # Création d'une colonne date propre (1er du mois)
    # Format actuel : quarter="2023Q1", month=1 -> "2023-01-01"
    df_wfp['year'] = df_wfp['quarter'].str[:4].astype(int)
    df_wfp['time'] = pd.to_datetime(df_wfp.assign(day=1)[['year', 'month', 'day']])
    
    # On garde ce qui nous intéresse
    # ADM1_NAME est le nom de l'état, inadequate est la cible
    df_wfp_clean = df_wfp[['time', 'ADM1_NAME', 'inadequate']].copy()
    
    # Renommage pour la fusion
    df_wfp_clean = df_wfp_clean.rename(columns={'ADM1_NAME': 'NAME_1'})
    
    print(f"   -> WFP prêt : {df_wfp_clean.shape} lignes.")
else:
    raise FileNotFoundError(f" Impossible de trouver {wfp_path}.")


Chargement des données WFP...
   -> WFP prêt : (1010, 3) lignes.


In [13]:

# ==============================================================================
# 4. FUSION FINALE (MERGE)
# ==============================================================================
print("Fusion des deux bases sur [Date, État]...")

# Inner join : on ne garde que les lignes où on a À LA FOIS la météo et la donnée WFP
df_final = pd.merge(df_wfp_clean, df_climate_admin1, on=['time', 'NAME_1'], how='inner')

# Sauvegarde
df_final.to_parquet(output_path, index=False)

print("-" * 30)
print(f"✅ SUCCÈS ! Dataset Maître créé : {output_path}")
print(f"Dimensions : {df_final.shape}")
print(f"Nombre d'États couverts : {df_final['NAME_1'].nunique()}")
print("-" * 30)

# Aperçu
display(df_final.head())

# Petite vérif visuelle : Corrélation brute
import seaborn as sns
import matplotlib.pyplot as plt

cols_corr = ['inadequate', 'Rainf_f_tavg', 'SoilMoi00_10cm_tavg', 'Tair_f_tavg']
corr = df_final[cols_corr].corr()
print("\nMatrice de corrélation rapide :")
display(corr.style.background_gradient(cmap='coolwarm'))

Fusion des deux bases sur [Date, État]...
------------------------------
✅ SUCCÈS ! Dataset Maître créé : ../data/processed/NIGERIA_WFP_ADMIN1.parquet
Dimensions : (552, 31)
Nombre d'États couverts : 24
------------------------------


,time,NAME_1,inadequate,Evap_tavg,LWdown_f_tavg,Lwnet_tavg,Psurf_f_tavg,Qair_f_tavg,Qg_tavg,Qh_tavg,...,Tair_f_tavg,Wind_f_tavg,SoilMoi00_10cm_tavg,SoilMoi10_40cm_tavg,SoilMoi40_100cm_tavg,SoilMoi100_200cm_tavg,SoilTemp00_10cm_tavg,SoilTemp10_40cm_tavg,SoilTemp40_100cm_tavg,SoilTemp100_200cm_tavg
0,2023-01-01,Abia,0.296774,0.000016,385.566324,-71.064699,99941.271400,0.013670,1.535858,52.585289,...,299.067207,3.369548,0.280146,0.364495,0.374807,0.389133,300.138219,299.605325,299.121555,298.700373
1,2023-01-01,Adamawa,0.340483,0.000006,327.309607,-120.776627,96184.946619,0.005087,2.256282,75.890667,...,298.876978,5.478152,0.151770,0.307479,0.318816,0.360368,299.263407,298.643008,298.328045,298.392429
2,2023-01-01,Bauchi,0.532895,0.000007,314.941737,-117.574605,95708.880950,0.004648,1.426913,59.048999,...,296.026673,6.130964,0.149414,0.321856,0.330437,0.358364,296.697754,296.368762,296.427992,297.035517
3,2023-01-01,Benue,0.356250,0.000004,353.904917,-101.338154,99314.368391,0.009095,2.423522,95.164377,...,298.811289,5.229476,0.166896,0.361930,0.365920,0.376692,300.209476,299.508505,298.974352,298.657582
4,2023-01-01,Borno,0.402923,0.000006,315.299290,-126.421503,97280.342111,0.003886,1.509641,56.107873,...,297.489843,7.072262,0.145651,0.324777,0.331708,0.354168,298.340759,298.009549,298.091304,298.766369



Matrice de corrélation rapide :


,inadequate,Rainf_f_tavg,SoilMoi00_10cm_tavg,Tair_f_tavg
inadequate,1.000000,-0.174587,-0.338349,0.034708
Rainf_f_tavg,-0.174587,1.000000,0.820007,-0.101381
SoilMoi00_10cm_tavg,-0.338349,0.820007,1.000000,-0.078784
Tair_f_tavg,0.034708,-0.101381,-0.078784,1.000000
